In [46]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime

from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
from google.oauth2 import service_account


# https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries
# https://codelabs.developers.google.com/codelabs/cloud-bigquery-python#0

# https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# https://cloud.google.com/python/docs/setup


# https://cloud.google.com/apis/docs/getting-started#enabling_apis



# https://cloud.google.com/bigquery/docs/reference/libraries
# https://cloud.google.com/python/docs/reference/bigquery/latest
# https://gcloud.readthedocs.io/en/latest/bigquery-client.html

In [47]:
# credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\xxxxxx.json')
# print(credentials.project_id)

In [48]:
projectId='pongthorn'
dataset_id='DemoSMartDW'
table_id = f"{projectId}.{dataset_id}.incident"
start_date_query='2020-01-01'

In [49]:
# client = bigquery.Client(credentials= credentials,project=projectId)
client = bigquery.Client(project=projectId)

In [50]:
try:
    client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
except NotFound:
    raise Exception("Table {} is not found.".format(table_id))

Table pongthorn.DemoSMartDW.incident already exists.


In [51]:
dt_imported=datetime.now()
str_imported=dt_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Imported DateTime: {str_imported}" )

Imported DateTime: 2023-04-20 16:18:56


In [52]:
sql_lastImport=f"SELECT max(imported_at) as last_imported from `{table_id}` where open_datetime>='{start_date_query}' "
job_lastImported=client.query(sql_lastImport)
str_lastImported=None
for row in job_lastImported:    
    if row.last_imported is not None: 
        str_lastImported=row.last_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Last Imported DateTime: {str_lastImported}" )

if str_lastImported is not None:
  start_date_query=str_lastImported

print(f"Start Import on update_at of last imported date : {start_date_query}" )

Last Imported DateTime: None
Start Import on update_at of last imported date : 2020-01-01


In [53]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
         database='SMartDB', user='postgres',
      password='P@ssw0rd', host='localhost', 
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [54]:
# check schema 
sql_incident="""

select
incident.id as id, incident.incident_no as incident_no,

severity.id as  severity_id,
severity.severity_name as  severity_name,

service_level.sla_name as sla,

product_type.productype_name as product_type,brand.brand_name as brand,

xtype.incident_type_name as incident_type,
status.incident_status_name as status,
service.service_type_name service_type,

model.model_name as model,CASE WHEN failure_type IS NULL THEN  0 ELSE 1 END AS is_failure_type,


(select count(*) from  app_incident_detail  as detail where  detail.incident_master_id=incident.id ) as count_detail


,TO_CHAR(incident.incident_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as open_datetime
,TO_CHAR(incident.incident_close_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as close_datetime

,TO_CHAR(incident.incident_problem_start  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as response_datetime
,TO_CHAR(incident.incident_problem_end  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as resolved_datetime

,company.company_name as company
,TO_CHAR(incident.updated_at,'YYYY-MM-DD HH24:MI:SS') as updated_at 


from app_incident as incident
inner join app_incident_type as  xtype on incident.incident_type_id = xtype.id
inner join  app_incident_status as status on incident.incident_status_id = status.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join  app_service_type as service on incident.service_type_id= service.id

inner join app_inventory as inventory on incident.inventory_id = inventory.id

inner join app_brand as brand on inventory.brand_id = brand.id
inner join app_model as model on inventory.model_id = model.id
inner join app_product_type as product_type on inventory.product_type_id = product_type.id
inner join app_sla as service_level on inventory.customer_sla_id = service_level.id

inner join app_project as project on inventory.project_id = project.id
inner join app_company as company on project.company_id = company.id

where incident.incident_status_id =4
and incident.updated_at>=%(start_date_param)s

order by incident.incident_datetime desc


"""


In [55]:
print("Create all issues dataframe")

dict_params={"start_date_param":start_date_query}
df_all=list_data(sql_incident,dict_params,get_postgres_conn())

if df_all.empty==True:
    print("no transsaction update")
    exit()


Create all issues dataframe


In [56]:
# convert object to datetime
dateTimeCols=['open_datetime','response_datetime','resolved_datetime','close_datetime','updated_at']
for col in dateTimeCols:
 df_all[col]=pd.to_datetime(df_all[col], format='%Y-%m-%d %H:%M',errors= 'coerce')

df_all['imported_at']=dt_imported


df_all.dropna(inplace=True)

#df_all=df_all.head(10)
print(df_all.info())
df_all.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2492 entries, 0 to 2492
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 2492 non-null   int64         
 1   incident_no        2492 non-null   object        
 2   severity_id        2492 non-null   int64         
 3   severity_name      2492 non-null   object        
 4   sla                2492 non-null   object        
 5   product_type       2492 non-null   object        
 6   brand              2492 non-null   object        
 7   incident_type      2492 non-null   object        
 8   status             2492 non-null   object        
 9   service_type       2492 non-null   object        
 10  model              2492 non-null   object        
 11  is_failure_type    2492 non-null   int64         
 12  count_detail       2492 non-null   int64         
 13  open_datetime      2492 non-null   datetime64[ns]
 14  close_da

,id,incident_no,severity_id,severity_name,sla,product_type,brand,incident_type,status,service_type,model,is_failure_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,company,updated_at,imported_at
0,2663,SR-ES-23-2663,2,Major,24x7 6Hrs Response Time,Server,HPE,General Incident,Closed,Incident,3PAR StoreServ 7400,0,3,2023-04-18 22:18:00,2023-04-18 23:20:00,2023-04-18 22:51:00,2023-04-18 23:20:00,INET- (HPE Product),2023-04-19 09:12:24,2023-04-20 16:18:56.683576
1,2651,SR-ES-23-2651,2,Major,24x7 4Hrs Resolution Time,Firewall,Palo Alto,General Incident,Closed,Incident,PA-5250,0,1,2023-04-16 07:25:00,2023-04-16 08:00:00,2023-04-16 07:26:00,2023-04-16 08:00:00,New-GFMIS,2023-04-18 08:48:17,2023-04-20 16:18:56.683576
2,2655,SR-ES-23-2655,3,Minor,24x7 4Hrs Resolution Time,Storage,NetApp,Hard Disk Drive Failure,Closed,Incident,FAS8020,0,2,2023-04-15 07:30:00,2023-04-15 15:30:00,2023-04-15 07:30:00,2023-04-15 15:30:00,INET-(NetApp and Non HPE),2023-04-18 10:37:35,2023-04-20 16:18:56.683576
3,2640,SR-ES-23-2640,2,Major,24x7 4Hrs Response Time,Server,HPE,Memory Failure,Closed,Incident,DL380 Gen10,1,2,2023-04-14 13:08:00,2023-04-17 14:49:00,2023-04-14 13:17:00,2023-04-14 22:00:00,Coforge (NIIT),2023-04-17 14:48:42,2023-04-20 16:18:56.683576
4,2656,SR-ES-23-2656,3,Minor,24x7 4Hrs Resolution Time,Storage,NetApp,Hard Disk Drive Failure,Closed,Incident,FAS8020,0,2,2023-04-14 06:30:00,2023-04-14 15:00:00,2023-04-14 06:31:00,2023-04-14 15:00:00,INET-(NetApp and Non HPE),2023-04-18 10:46:45,2023-04-20 16:18:56.683576
5,2642,SR-ES-23-2642,2,Major,24x7 4Hrs Resolution Time,Storage,NetApp,General Incident,Closed,Incident,SG5760,0,3,2023-04-13 12:45:00,2023-04-17 11:30:00,2023-04-13 12:49:00,2023-04-17 11:30:00,BDMS,2023-04-17 22:13:00,2023-04-20 16:18:56.683576
6,2650,SR-ES-23-2650,2,Major,24x7 4Hrs Resolution Time,Firewall,Palo Alto,General Incident,Closed,Incident,PA-5250,0,1,2023-04-13 07:18:00,2023-04-13 08:00:00,2023-04-13 07:20:00,2023-04-13 08:00:00,New-GFMIS,2023-04-18 08:43:06,2023-04-20 16:18:56.683576
7,2645,SR-ES-23-2645,2,Major,24x7 4Hrs Response Time,Hardware,HPE,General Incident,Closed,Incident,MSL 4048,0,1,2023-04-12 19:50:00,2023-04-12 23:59:00,2023-04-12 20:00:00,2023-04-12 23:59:00,TLI,2023-04-17 23:09:14,2023-04-20 16:18:56.683576
8,2646,SR-ES-23-2646,2,Major,24x7 4Hrs Response Time,Software,Veritas,Software,Closed,Incident,Veritas Netbackup,0,1,2023-04-12 19:50:00,2023-04-13 03:00:00,2023-04-12 20:00:00,2023-04-13 03:00:00,TLI,2023-04-17 23:18:00,2023-04-20 16:18:56.683576
9,2641,SR-ES-23-2641,2,Major,24x7 4Hrs Resolution Time,Software,VMWare,Software,Closed,Incident,VMWARE NSX DATA CENTER ADVANCED PER PROCESSOR,0,1,2023-04-12 15:29:00,2023-04-12 22:00:00,2023-04-12 15:36:00,2023-04-12 22:00:00,BDMS,2023-04-17 21:56:39,2023-04-20 16:18:56.683576


In [57]:
def insertDataFrameToBQ(df_trasns):
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            df_trasns, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_trasns), "Imported closed incident to bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        print(e) 

try:
    insertDataFrameToBQ(df_all)
except Exception as ex:
    raise ex




Total  2492 Imported closed incident to bigquery successfully


In [58]:
importDate=dt_imported.strftime("%Y-%m-%d")

def load_data_bq(sql:str):
 query_result=client.query(sql)
 dfx=query_result.to_dataframe()
 return dfx
sql_query=f""" SELECT * FROM `{table_id}` WHERE imported_at >= '{importDate}' LIMIT 10 """
df=load_data_bq(sql_query)
df

,id,incident_no,severity_id,severity_name,sla,product_type,brand,model,incident_type,status,service_type,is_failure_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,company,updated_at,imported_at
0,1520,SR-ES-22-1520,1,Critical,24x7 6Hrs Resolution Time,Storage,YIP,dummy,Controller/Node Failure,Closed,Incident,1,1,2022-09-27 12:32:00,2022-09-28 11:43:00,2022-09-27 12:40:00,2022-09-27 14:00:00,PG,2022-09-28 11:48:54,2023-04-20 16:18:56.683576
1,619,SR-ES-22-0619,1,Critical,24x7 4Hrs Resolution Time,Storage,NetApp,FAS3240,Controller/Node Failure,Closed,Incident,0,2,2022-05-06 21:30:00,2022-05-07 05:00:00,2022-05-06 21:30:00,2022-05-07 05:00:00,MEA (ฝมธ.),2022-05-09 14:31:31,2023-04-20 16:18:56.683576
2,617,SR-ES-22-0617,1,Critical,24x7 4Hrs Resolution Time,Storage,NetApp,FAS3240,General Incident,Closed,Incident,0,2,2022-05-06 18:30:00,2022-05-07 05:00:00,2022-05-06 21:30:00,2022-05-07 05:00:00,MEA (ฝมธ.),2022-05-09 12:30:59,2023-04-20 16:18:56.683576
3,526,SR-ES-22-0526,1,Critical,24x7 4Hrs Resolution Time,Storage,NetApp,FAS8020,Controller/Node Failure,Closed,Incident,0,2,2022-04-29 01:57:00,2022-05-29 14:00:00,2022-04-29 02:09:00,2022-04-29 02:30:00,INET-(NetApp and Non HPE),2022-05-10 14:30:41,2023-04-20 16:18:56.683576
4,26,SR-ES-22-0026,1,Critical,24x7 4Hrs Resolution Time,Storage,NetApp,FAS8020,Hard Disk Drive Failure,Closed,Incident,0,1,2022-03-26 17:45:00,2022-03-27 22:00:00,2022-03-26 17:50:00,2022-03-27 22:00:00,KTBCS,2022-03-28 11:42:18,2023-04-20 16:18:56.683576
5,32,SR-ES-22-0032,1,Critical,24x7 6Hrs Resolution Time,Storage,NetApp,FAS8020,Power Supply Failure,Closed,Incident,0,1,2022-03-17 22:17:00,2022-03-21 14:28:00,2022-03-17 23:56:00,2022-03-21 14:28:00,AIS,2022-03-28 21:45:37,2023-04-20 16:18:56.683576
6,300,SR-ES-22-0300,1,Critical,24x7 6Hrs Resolution Time,Storage,NetApp,FAS8020,General Incident,Closed,Incident,0,1,2022-02-08 17:53:00,2022-04-01 09:50:00,2022-02-08 18:35:00,2022-02-09 21:10:00,AIS,2022-04-11 14:46:50,2023-04-20 16:18:56.683576
7,163,SR-ES-22-0163,1,Critical,24x7 4Hrs Resolution Time,Storage,NetApp,FAS8020,Hard Disk Drive Failure,Closed,Incident,0,0,2021-12-28 01:35:00,2021-12-28 10:22:00,2021-12-28 01:41:00,2021-12-28 10:22:00,KTBCS,2022-04-04 11:23:01,2023-04-20 16:18:56.683576
8,614,SR-ES-22-0614,1,Critical,24x7 4Hrs Resolution Time,Storage,NetApp,FAS8300,General Incident,Closed,Incident,0,1,2022-05-06 21:16:00,2022-05-06 23:17:00,2022-05-06 21:23:00,2022-05-06 23:17:00,KTBCS,2022-05-10 00:10:43,2023-04-20 16:18:56.683576
9,1754,SR-ES-22-1754,1,Critical,24x7 4Hrs Resolution Time,Firewall,Palo Alto,PA-5250,General Incident,Closed,Incident,0,1,2022-11-01 14:26:00,2022-11-01 14:37:00,2022-11-01 14:27:00,2022-11-01 14:27:00,New-GFMIS,2022-11-01 15:05:10,2023-04-20 16:18:56.683576
